# 监督学习在量化中的应用

## 📋 概述

监督学习是量化交易中应用最广泛的机器学习方法，通过使用历史数据训练模型来预测股票价格或收益。本章节将详细介绍监督学习在量化交易中的应用，包括分类问题（涨跌预测）、回归问题（价格预测）以及常用算法。

**学习方式**：本文件是Jupyter Notebook格式，你可以边看边运行代码，通过实际代码示例理解监督学习在量化中的应用。


In [ ]:
# 安装必要的库（如果还没有安装）
# !pip install pandas numpy matplotlib scikit-learn

# 导入库
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境准备完成！")


## 📖 核心内容

### 1. 监督学习在量化中的应用场景

#### 1.1 分类问题

**涨跌预测**：
- 预测股票未来涨跌方向
- 二分类：上涨/下跌
- 多分类：大涨/小涨/持平/小跌/大跌

**信号生成**：
- 买入/卖出/持有信号
- 基于模型预测生成交易信号

#### 1.2 回归问题

**价格预测**：
- 预测未来价格
- 预测收益率
- 预测波动率

**因子预测**：
- 预测财务指标
- 预测技术指标

让我们创建一个完整的监督学习示例：


In [ ]:
# 生成示例数据
np.random.seed(42)
n_days = 500
dates = pd.date_range(start='2023-01-01', periods=n_days, freq='D')
dates = dates[dates.weekday < 5]

# 生成价格数据
prices = 10 + np.cumsum(np.random.randn(len(dates)) * 0.1)
volumes = np.random.randint(1000000, 10000000, len(dates))

price_data = pd.DataFrame({
    'Open': prices + np.random.randn(len(dates)) * 0.05,
    'High': prices + np.abs(np.random.randn(len(dates)) * 0.1),
    'Low': prices - np.abs(np.random.randn(len(dates)) * 0.1),
    'Close': prices + np.random.randn(len(dates)) * 0.05,
    'Volume': volumes
}, index=dates)

price_data['High'] = price_data[['Open', 'Close', 'High']].max(axis=1)
price_data['Low'] = price_data[['Open', 'Close', 'Low']].min(axis=1)

# 计算特征
price_data['Returns'] = price_data['Close'].pct_change()
price_data['MA5'] = price_data['Close'].rolling(window=5).mean()
price_data['MA20'] = price_data['Close'].rolling(window=20).mean()
price_data['Volatility'] = price_data['Returns'].rolling(window=20).std()
price_data['Volume_Ratio'] = price_data['Volume'] / price_data['Volume'].rolling(window=20).mean()
price_data['Momentum'] = price_data['Close'] / price_data['Close'].shift(5) - 1

# 计算目标变量（未来1期收益率）
price_data['Future_Return'] = price_data['Returns'].shift(-1)

# 创建分类标签（涨跌预测）
price_data['Direction'] = (price_data['Future_Return'] > 0).astype(int)

# 删除缺失值
price_data = price_data.dropna()

print(f"数据准备完成！共 {len(price_data)} 条记录")
print(f"\n特征列：")
feature_cols = ['Returns', 'MA5', 'MA20', 'Volatility', 'Volume_Ratio', 'Momentum']
print(feature_cols)
print(f"\n目标变量：")
print(f"分类：Direction (涨跌方向)")
print(f"回归：Future_Return (未来收益率)")


In [ ]:
# 准备分类数据
X_class = price_data[feature_cols].values
y_class = price_data['Direction'].values

# 划分训练集和测试集
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.3, random_state=42, shuffle=False
)

# 标准化特征
scaler_class = StandardScaler()
X_train_class_scaled = scaler_class.fit_transform(X_train_class)
X_test_class_scaled = scaler_class.transform(X_test_class)

# 训练随机森林分类器
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42, max_depth=5)
rf_classifier.fit(X_train_class_scaled, y_train_class)

# 训练逻辑回归分类器
lr_classifier = LogisticRegression(random_state=42, max_iter=1000)
lr_classifier.fit(X_train_class_scaled, y_train_class)

# 预测
rf_pred = rf_classifier.predict(X_test_class_scaled)
lr_pred = lr_classifier.predict(X_test_class_scaled)

# 评估
rf_accuracy = accuracy_score(y_test_class, rf_pred)
lr_accuracy = accuracy_score(y_test_class, lr_pred)

print("分类模型结果：")
print(f"随机森林准确率：{rf_accuracy:.4f}")
print(f"逻辑回归准确率：{lr_accuracy:.4f}")

print("\n随机森林分类报告：")
print(classification_report(y_test_class, rf_pred, target_names=['下跌', '上涨']))

# 可视化预测结果
fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

# 实际涨跌
test_dates = price_data.index[-len(y_test_class):]
axes[0].plot(test_dates, y_test_class, 'o-', label='实际', linewidth=1, markersize=4, alpha=0.7)
axes[0].plot(test_dates, rf_pred, 's-', label='随机森林预测', linewidth=1, markersize=4, alpha=0.7)
axes[0].plot(test_dates, lr_pred, '^-', label='逻辑回归预测', linewidth=1, markersize=4, alpha=0.7)
axes[0].set_title('涨跌预测结果', fontsize=14, fontweight='bold')
axes[0].set_ylabel('涨跌 (0=跌, 1=涨)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 预测概率（随机森林）
rf_proba = rf_classifier.predict_proba(X_test_class_scaled)[:, 1]
axes[1].plot(test_dates, rf_proba, linewidth=2, label='上涨概率', color='green', alpha=0.7)
axes[1].axhline(y=0.5, color='red', linestyle='--', linewidth=1, label='阈值0.5')
axes[1].fill_between(test_dates, 0.5, rf_proba, where=(rf_proba > 0.5), 
                     alpha=0.3, color='green', label='预测上涨区域')
axes[1].fill_between(test_dates, 0.5, rf_proba, where=(rf_proba <= 0.5), 
                     alpha=0.3, color='red', label='预测下跌区域')
axes[1].set_title('预测概率', fontsize=14, fontweight='bold')
axes[1].set_xlabel('日期')
axes[1].set_ylabel('上涨概率')
axes[1].set_ylim(0, 1)
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


### 3. 回归问题：收益率预测

让我们使用随机森林和线性回归来预测未来收益率：


In [ ]:
# 准备回归数据
X_reg = price_data[feature_cols].values
y_reg = price_data['Future_Return'].values

# 划分训练集和测试集
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=42, shuffle=False
)

# 标准化特征
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

# 训练随机森林回归器
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=5)
rf_regressor.fit(X_train_reg_scaled, y_train_reg)

# 训练线性回归
lr_regressor = LinearRegression()
lr_regressor.fit(X_train_reg_scaled, y_train_reg)

# 预测
rf_pred_reg = rf_regressor.predict(X_test_reg_scaled)
lr_pred_reg = lr_regressor.predict(X_test_reg_scaled)

# 评估
rf_mse = mean_squared_error(y_test_reg, rf_pred_reg)
rf_r2 = r2_score(y_test_reg, rf_pred_reg)
lr_mse = mean_squared_error(y_test_reg, lr_pred_reg)
lr_r2 = r2_score(y_test_reg, lr_pred_reg)

print("回归模型结果：")
print(f"随机森林 - MSE: {rf_mse:.6f}, R²: {rf_r2:.4f}")
print(f"线性回归 - MSE: {lr_mse:.6f}, R²: {lr_r2:.4f}")

# 可视化预测结果
fig, axes = plt.subplots(2, 1, figsize=(15, 10), sharex=True)

test_dates_reg = price_data.index[-len(y_test_reg):]

# 预测值对比
axes[0].plot(test_dates_reg, y_test_reg * 100, 'o-', label='实际收益率', 
            linewidth=1.5, markersize=4, alpha=0.7, color='black')
axes[0].plot(test_dates_reg, rf_pred_reg * 100, 's-', label='随机森林预测', 
            linewidth=1.5, markersize=4, alpha=0.7, color='green')
axes[0].plot(test_dates_reg, lr_pred_reg * 100, '^-', label='线性回归预测', 
            linewidth=1.5, markersize=4, alpha=0.7, color='blue')
axes[0].axhline(y=0, color='red', linestyle='--', linewidth=0.5)
axes[0].set_title('收益率预测结果', fontsize=14, fontweight='bold')
axes[0].set_ylabel('收益率 (%)')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# 预测误差
rf_error = (rf_pred_reg - y_test_reg) * 100
lr_error = (lr_pred_reg - y_test_reg) * 100

axes[1].plot(test_dates_reg, rf_error, 's-', label='随机森林误差', 
            linewidth=1, markersize=3, alpha=0.7, color='green')
axes[1].plot(test_dates_reg, lr_error, '^-', label='线性回归误差', 
            linewidth=1, markersize=3, alpha=0.7, color='blue')
axes[1].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
axes[1].set_title('预测误差', fontsize=14, fontweight='bold')
axes[1].set_xlabel('日期')
axes[1].set_ylabel('误差 (%)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# 特征重要性
rf_feature_importance = rf_classifier.feature_importances_
rf_reg_feature_importance = rf_regressor.feature_importances_

# 可视化
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 分类模型特征重要性
indices_class = np.argsort(rf_feature_importance)[::-1]
axes[0].barh(range(len(feature_cols)), rf_feature_importance[indices_class], 
            color='skyblue', alpha=0.7, edgecolor='black')
axes[0].set_yticks(range(len(feature_cols)))
axes[0].set_yticklabels([feature_cols[i] for i in indices_class])
axes[0].set_xlabel('重要性')
axes[0].set_title('分类模型特征重要性', fontsize=12, fontweight='bold')
axes[0].grid(True, alpha=0.3, axis='x')

# 回归模型特征重要性
indices_reg = np.argsort(rf_reg_feature_importance)[::-1]
axes[1].barh(range(len(feature_cols)), rf_reg_feature_importance[indices_reg], 
            color='lightgreen', alpha=0.7, edgecolor='black')
axes[1].set_yticks(range(len(feature_cols)))
axes[1].set_yticklabels([feature_cols[i] for i in indices_reg])
axes[1].set_xlabel('重要性')
axes[1].set_title('回归模型特征重要性', fontsize=12, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n特征重要性排名（分类模型）：")
for i, idx in enumerate(indices_class, 1):
    print(f"{i}. {feature_cols[idx]}: {rf_feature_importance[idx]:.4f}")

print("\n特征重要性排名（回归模型）：")
for i, idx in enumerate(indices_reg, 1):
    print(f"{i}. {feature_cols[idx]}: {rf_reg_feature_importance[idx]:.4f}")


## 💡 关键要点总结

1. **应用场景**：分类（涨跌预测）、回归（收益率预测）
2. **常用算法**：随机森林、逻辑回归、线性回归
3. **特征工程**：技术指标、财务指标、另类数据
4. **模型评估**：准确率、MSE、R²、特征重要性

## 🛠️ 工具与软件

### 机器学习工具

- **scikit-learn**：Python机器学习库
- **pandas/numpy**：数据处理
- **自定义工具**：根据需求实现

### 注意事项

- ✅ **特征选择**：选择有预测能力的特征
- ✅ **数据预处理**：标准化、缺失值处理
- ✅ **模型验证**：样本外测试、交叉验证
- ⚠️ **过拟合风险**：避免过度拟合历史数据
- ⚠️ **数据泄露**：避免使用未来信息

## 🔗 相关知识点

- [深度学习在量化中的应用](./深度学习在量化中的应用.md)
- [强化学习在量化中的应用](./强化学习在量化中的应用.md)
- [因子挖掘方法](../04_因子分析/因子挖掘方法.ipynb)

## 📚 拓展阅读

- 《机器学习》- 周志华
- 《Python机器学习》- 深入理解机器学习
- 量化交易相关论文 - 最新研究方法
